# 벽돌 깨기 게임
- 중요한 포인트 4가지
    1. No-operation ==> Breakout 실행 환경을 새로 시작했을 때, 이후 30단계는 아무행동도 취하지 않는다. 이유는 게임을 시작한 후 초기 상태를 다양하게 만들어 특정한 상태에만 특화된 학습이 일어나지 않도록 하기 위해서이다.
    2. Episodic Life ==> 이 게임은 목숨이 5개이다. 게임 중에 실패를 허용하는 것은 학습하기 어려우므로 한번만 실패해도 게임이 종료되도록 한다. 또한 매번 게임을 완전히 다시 시작하면 초기 상태만 학습이 되므로 다양한 상태를 학습할 수 없다. 따라서 목숨이 날아가더라도 초기화되신 블록이 부서진 채로 다시 시작하게 된다.
    3. Max and Skip ==> 이 게임은 60Hz로 진행된다. 속도가 너무 빠르기 때문에 4프레임마다 행동을 판단하고 행동은 4프레임 동안 지속시킨다. 따라서 에이전트의 행동은 15Hz가 된다. 아타리 게임 중에서는 홀수 프레임과 짝수 프레임에 출력되는 내용이 서로 다른 게임이 있으므로 여기서도 3번째 프레임과 4번째 프레임 이미지의 최댓 값으로 만든 이미지를 판단에 사용한다.
    4. Wrap frame ==> 이 게임의 이미지는 (210, 160, 3)으로 되어 있다. 이 이미지를 논문의 구현에 맞춰 가로세로 84픽셀 그레이스케일로 변환한다.    

In [83]:
import numpy as np
from collections import deque
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
from gym import spaces
from gym.spaces.box import Box

In [84]:
import cv2
cv2.ocl.setUseOpenCL(False)

class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        '''첫 번째 트릭 No-Operation. 초기화 후 일정 단계에 이를때까지 아무 행동도 하지않고
        게임 초기 상태를 다양하게 하여 특정 시작 상태만 학습하는 것을 방지한다'''

        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.randint(
                1, self.noop_max + 1)  # pylint: disable=E1101
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)


class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        '''두 번째 트릭 Episodic Life. 한번 실패를 게임 종료로 간주하나, 다음 게임을 같은 블록 상태로 시작'''
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condtion for a few frames
            # so its important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        '''5번 실패하면 게임을 완전히 다시 시작'''
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs


class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        '''세 번째 트릭 Max and Skip. 4프레임 동안 같은 행동을 지속하되, 3번째와 4번째 프레임의 최댓값 이미지를 관측 obs로 삼는다'''
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros(
            (2,)+env.observation_space.shape, dtype=np.uint8)
        self._skip = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2:
                self._obs_buffer[0] = obs
            if i == self._skip - 1:
                self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)


class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env):
        '''네 번째 트릭 Warp frame. DQN 네이처 논문 구현과 같이 84*84 흑백 이미지를 사용'''
        gym.ObservationWrapper.__init__(self, env)
        self.width = 84
        self.height = 84
        self.observation_space = spaces.Box(low=0, high=255,
                                            shape=(self.height, self.width, 1), dtype=np.uint8)

    def observation(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height),
                           interpolation=cv2.INTER_AREA)
        return frame[:, :, None]


class WrapPyTorch(gym.ObservationWrapper):
    def __init__(self, env=None):
        '''인덱스 순서를 파이토치 미니배치와 같이 조정하는 래퍼'''
        super(WrapPyTorch, self).__init__(env)
        obs_shape = self.observation_space.shape
        self.observation_space = Box(
            self.observation_space.low[0, 0, 0],
            self.observation_space.high[0, 0, 0],
            [obs_shape[2], obs_shape[1], obs_shape[0]],
            dtype=self.observation_space.dtype)

    def observation(self, observation):
        return observation.transpose(2, 0, 1)

In [85]:
# 실행환경 생성함수
# 병렬 실행환경
from stable_baselines3.common.vec_env import DummyVecEnv
def make_env(env_id, seed, rank):
    def _thunk():
        env = gym.make(env_id)
        env = NoopResetEnv(env, noop_max=30)
        env = MaxAndSkipEnv(env, skip=4)
        env.seed(seed + rank) # 난수 시드 설정
        env = EpisodicLifeEnv(env)
        env = WarpFrame(env)
        env = WrapPyTorch(env)
        return env
    return _thunk

In [86]:
ENV_NAME = "BreakoutNoFrameskip-v4"
# v0는 2~4개 프레임을 자동으로 생략하므로 이 기능이 없는 버전을 사용한다.
NUM_SKIP_FRAME = 4 # 생략할 프레임 수
NUM_STACK_FRAME = 4 # 하나의 상태로 사용할 프레임 수
NOOP_MAX = 30 # 초기화후 No-operation을 적용할 최초 프레임 수의 최댓값
NUM_PROCESSES = 16 # 병렬로 실행할 프로세스 수
NUM_ADVANCED_STEP = 5 # Advanced 학습할 단계 수
GAMMA = 0.99 # 시간 학습률

TOTAL_FRAMES = 10e6 # 학습에 사용되는 프레임 수
NUM_UPDATES = int(TOTAL_FRAMES / NUM_ADVANCED_STEP / NUM_PROCESSES) # 신경망의 수정 총 횟수, 약 125,000이 됨

# A2C 손실 함수를 계산하기 위한 상수
value_loss_coef = 0.5
entropy_coef = 0.01
max_grad_norm = 0.5

# 최적화 기법 RMSprop에 대한 설정
lr = 7e-4
eps = 1e-5
alpha = 0.99

In [87]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [88]:
# 메모리 클래스 정의
class RolloutStorage(object):
    def __init__(self, num_steps, num_processes, obs_shape):
        self.observations = torch.zeros(num_steps + 1, num_processes, *obs_shape).to(device)
        self.masks = torch.ones(num_steps + 1, num_processes, 1).to(device) # 해당 단계가 에피소드의 끝인지 여부를 표시하기 위한 변수 다음 단계가 존재하면 1 아니면 0값을 가짐
        self.rewards = torch.zeros(num_steps, num_processes, 1).to(device)
        self.actions = torch.zeros(num_steps, num_processes, 1).long().to(device) # long() 형으로 변환
        # 할인 총 보상 저장
        self.returns = torch.zeros(num_steps + 1, num_processes, 1).to(device)
        self.index = 0 # insert할 인덱스
        
    def insert(self, current_obs, action, reward, mask):
        # 이 함수에서 아래의 코드는 각 단계에서 얻은 값들을 복사하여 저장을 한다.
        # 예를 들어 actions의 shape : (num_steps, num_processes, 1)인데 이를 (index, num_processes, 1)처럼 접근하여 저장을 하겠다는 의미이다.
        self.observations[self.index + 1].copy_(current_obs) # 현재 상태에서 어떤 행동을 취했을 때 다음 상태 저장
        self.masks[self.index + 1].copy_(mask) # 에피소드가 끝났는지 확인
        self.rewards[self.index].copy_(reward) # 현재 상태에서 얻음 보상
        self.actions[self.index].copy_(action) # 현재 상태에서 취한 행동
        
        self.index = (self.index + 1) % NUM_ADVANCED_STEP
        
    def after_update(self): 
        # Advantage만큼 학습 단계가 진행되면 가장 새로운 transition을 index0에 저장
        self.observations[0].copy_(self.observations[-1]) # (5, 1, 2, 3, 4)
        self.masks[0].copy_(self.masks[-1]) # (5, 1, 2, 3, 4)
        
    def compute_returns(self, next_value):
        # 주의 : 5번쩨 단계 부터 거슬러 올라오며 계산
        # 주의 : 5번째 단계가 Advantage1, 4번째 단계는 Advantage2가 됨
        self.returns[-1] = next_value
        for ad_step in reversed(range(self.rewards.size(0))):
            self.returns[ad_step] = self.returns[ad_step + 1] * GAMMA * self.masks[ad_step + 1] + self.rewards[ad_step]

In [105]:
# A2C 신경망 구성
def init(module, gain):
    # 정확한 해를 찾기 위해 Orthogonal Matrix를 생성한다.
    # Orthogonal Matrix는 자기자신 행렬과 전치행렬을 곱했을 때 단위 행렬이 되는 행렬을 의미한다.
    nn.init.orthogonal_(module.weight.data, gain=gain)
    nn.init.constant_(module.bias.data, 0)
    return module

class Flatten(nn.Module):
    '''합성곱층의 출력 이미지를 1차원으로 변환하는 층'''
    def forward(self, x):
        return x.reshape(x.size(0), -1)

class Net(nn.Module):
    def __init__(self, n_out):
        super(Net, self).__init__()
        
        # 결합 가중치 초기화함수
        def init_(module): return init(module, gain=nn.init.calculate_gain('relu'))
        
        # 합성곱층을 정의
        self.conv = nn.Sequential(
            # 이미지 크기의 변화1 : (84, 84) ==> (20, 20)처럼 나오는 이유는 input 크기가 84인데 kernel_size가 8이고 stride가 4이므로 
            # 4x + 8 = 84를 구하면 x는 19가 나온다. 따라서 처음 값 1번을 더하면 20이 나온다. 출력인 32는 fileter수와 동일하다.
            # 이미지 크기의 변화2 : (20, 20) ==> (9, 9)
            # 앞서 계산 한것 처럼 2x + 4 = 20이므로 x = 8이 나오고 처음 횟수 1번을 더하면 9가 나온다. fileter의 수는 64이다.
            # 이미지 크기의 변화3: (9, 9) ==> (7, 7) 
            # 앞서 계산한 것처럼 x + 3 = 9이므로 x = 6이 되고 처음 횟수 1번을 더하면 7이 나온다. fileter의 수는 64이다.
            init_(nn.Conv2d(NUM_STACK_FRAME, 32, kernel_size=8, stride=4)), # 이미지 크기의 변화 (84*84 => 20 * 20)
            nn.ReLU(),
            init_(nn.Conv2d(32, 64, kernel_size=4, stride=2)),
            nn.ReLU(),
            init_(nn.Conv2d(64, 64, kernel_size=3, stride=1)),
            nn.ReLU(),
            Flatten(),
            init_(nn.Linear(64 * 7 * 7, 512)), # filter의 수는 64이고 이미지의 크기는 7*7이다.
            nn.ReLU())
        
        # 결합 가중치 초기화 함수
        def init_(module): return init(module, gain=1.0)
        
        # Critic을 정의
        self.critic = init_(nn.Linear(512, 1)) # 출력은 상태가치이므로 1개
        
        # 결합 가중치 초기화 함수
        def init_(module): return init(module, gain=0.01)
        
        # Actor를 정의
        self.actor = init_(nn.Linear(512, n_out)) # 출력이 행동이므로 출력 수는 행동의 가짓수
        
        self.train()
    
    def forward(self, x):
        inputs = x / 255.0
        conv_output = self.conv(inputs)
        critic_output = self.critic(conv_output)
        actor_output = self.actor(conv_output)
        
        return critic_output, actor_output
    
    def act(self, x):
        value, actor_output = self(x)
        probs = F.softmax(actor_output, dim=1)
        action = probs.multinomial(num_samples=1)
        return action
    
    def get_value(self, x):
        value, _ = self(x)
        return value
    
    def evaluate_actions(self, x, actions):
        value, actor_output = self(x)
        
        log_probs = F.log_softmax(actor_output, dim=1)
        action_log_probs = log_probs.gather(1, actions)
        
        probs = F.softmax(actor_output, dim=1)
        dist_entropy = -(log_probs * probs).sum(-1).mean()
        
        return value, action_log_probs, dist_entropy

In [106]:
class Brain(object):
    def __init__(self, actor_critic):
        self.actor_critic = actor_critic
        self.optimizer = optim.RMSprop(actor_critic.parameters(), lr=lr, eps=eps, alpha=alpha)
        
    def update(self, rollouts):
        obs_shape = rollouts.observations.size()[2:]
        num_steps = NUM_ADVANCED_STEP
        num_processes = NUM_PROCESSES
        
        # Tensor의 크기에 *을 붙이면 unpack되어 크기를 얻을 수 있다.
        values, action_log_probs, dist_entropy = self.actor_critic.evaluate_actions(
            rollouts.observations[:-1].reshape(-1, *obs_shape),
            rollouts.actions.reshape(-1, 1))
        
        values = values.reshape(num_steps, num_processes, 1) # torch.Size([5, 16, 1])
        action_log_probs = action_log_probs.reshape(num_steps, num_processes, 1)
        
        advantages = rollouts.returns[:-1] - values
        value_loss = advantages.pow(2).mean()
        
        # Advantages는 detach()해서 정수로 취급한다.
        action_gain = (advantages.detach() * action_log_probs).mean()
        total_loss = (value_loss * value_loss_coef - action_gain - dist_entropy * entropy_coef)
        nn.utils.clip_grad_norm_(self.actor_critic.parameters(), max_grad_norm)
        # 한번에 결합 가중치가 너무 크게 변화하지 않도록 경사의 최댓값ㅇ르 0.5로 제한한다.
        
        self.optimizer.step() # 결합 가중치 수정

In [109]:
class Environment:
    def run(self):
        seed_num = 1
        torch.manual_seed(seed_num)
        if use_cuda:
            torch.cuda.manual_seed(seed_num)
            
        # 실행환경 구축
        torch.set_num_threads(seed_num)
        envs = [make_env(ENV_NAME, seed_num, i) for i in range(NUM_PROCESSES)]
        envs = DummyVecEnv(envs) # 멀티프로세스 실행환경
        
        # 모든 에이전트가 공유하는 두뇌 역할 클래스 Brain객체 생성
        n_out = envs.action_space.n # 행동의 가짓수는 4개
        actor_critic = Net(n_out).to(device) # GPU사용
        global_brain = Brain(actor_critic)
        
        # 정보 저장용 변수 생성
        obs_shape = envs.observation_space.shape # (1, 84, 84)
        obs_shape = (obs_shape[0] * NUM_STACK_FRAME, *obs_shape[1:]) # (4, 84, 84)
        current_obs = torch.zeros(NUM_PROCESSES, *obs_shape).to(device) # (16, 4, 84, 84)
        rollouts = RolloutStorage(NUM_ADVANCED_STEP, NUM_PROCESSES, obs_shape) # rollout객체 생성
        episode_rewards = torch.zeros([NUM_PROCESSES, 1])
        final_rewards = torch.zeros([NUM_PROCESSES, 1])
        
        # 초기 상태로 시작
        obs = envs.reset()
        obs = torch.from_numpy(obs).float() # torch.Size([16, 1, 84, 84])
        current_obs[:, -1:] = obs # 제일 마지막 4번째 프레임에 가장 최근 관측 결과인 obs를 넣음
        
        # Advanted 핛브에 사용할 객체인 rollouts에 첫번째 상태로 현재 상태를 저장
        rollouts.observations[0].copy_(current_obs)
        
        #주 반복문
        for j in tqdm(range(NUM_UPDATES)):
            for step in range(NUM_ADVANCED_STEP):
                # 행동을 결정
                with torch.no_grad():
                    action = actor_critic.act(rollouts.observations[step])
                cpu_actions = action.squeeze(1).cpu().numpy()
                
                # 1단계를 병렬로 실행, 반환 값 obs의 크기는 (16, 1, 84, 84)
                obs, reward, done, info = envs.step(cpu_actions)
                
                # 보상을 텐서로 변환한 다음 에피소드 총 보상에 더함
                # 크기가 (16,) ==> (16, 1)로 변경
                reward = np.expand_dims(np.stack(reward), 1)
                reward = torch.from_numpy(reward).float()
                episode_rewards += reward
                
                # 각 프로세스 마다 done이 True면 0, False면 1
                masks = torch.FloatTensor([[0.0] if done_ else [1.0] for done_ in done])
                
                # 마지막 에피소드의 총 보상을 업데이트
                final_rewards *= masks
                final_rewards += (1 - masks) * episode_rewards
                
                # episode의 총 보상을 업데이트
                episode_rewards *= masks 
                
                masks = masks.to(device) # masks를 GPU로 보냄
                
                # masks의 크기를 torch.Size([16, 1]) ==> torch.Size([16, 1, 1, 1])로 변경하고 곱함
                current_obs *= masks.unsqueeze(2).unsqueeze(2)
                
                # 프레임을 모음
                # torch.Size([16, 1, 84, 84])
                obs = torch.from_numpy(obs).float()
                current_obs[:, :-1] = current_obs[:, 1:] # 0~2번째 프레임을 1~3번째 프레임으로 덮어씀
                current_obs[:, -1:] = obs # 4번째 프레임에 가장 최근 obs를 저장
                
                # 메모리 객체에 현 단계의 transition을 저장
                rollouts.insert(current_obs, action.data, reward, masks)
            # Advaned 학습의 for 문 끝
            
            # advanced학습 대상 단계 중 마지막 단계의 상태에서 예상되는 상태 가치를 계산
            with torch.no_grad():
                next_value = actor_critic.get_value(rollouts.observations[-1]).detach()
            # 모든 단계의 할인 총 보상을 계산하고, rollouts의 변수 returns를 업데이트
            rollouts.compute_returns(next_value)
            
            # 신경망 수정 및 rollout 업데이트
            global_brain.update(rollouts)
            rollouts.after_update()
            
            if j % 100 == 0:
                print("finished frames {}, mean/median reward {:.1f}/{:.1f}, min/max reward {:.1f}/{:.1f}".format(
                j*NUM_PROCESSES*NUM_ADVANCED_STEP,
                final_rewards.mean(),
                final_rewards.median(),
                final_rewards.min(),
                final_rewards.max()))
                
            if j % 12500 == 0:
                torch.save(global_brain.actor_critic.state_dict(), 'weight_'+str(j)+'.pth')
        
        torch.save(global_brain.actor_critic.state_dict(), 'weight_end.pth')

In [110]:
breakout_env = Environment()
breakout_env.run()

  0%|          | 2/125000 [00:00<4:28:00,  7.77it/s]

finished frames 0, mean/median reward 0.0/0.0, min/max reward 0.0/0.0


  0%|          | 102/125000 [00:11<3:31:13,  9.86it/s]

finished frames 8000, mean/median reward 0.4/0.0, min/max reward 0.0/2.0


  0%|          | 202/125000 [00:21<3:51:10,  9.00it/s]

...

100%|█████████▉| 124802/125000 [3:43:31<00:22,  8.71it/s]

finished frames 9984000, mean/median reward 0.1/0.0, min/max reward 0.0/1.0


100%|█████████▉| 124902/125000 [3:43:42<00:11,  8.90it/s]

finished frames 9992000, mean/median reward 0.2/0.0, min/max reward 0.0/1.0


100%|██████████| 125000/125000 [3:43:52<00:00,  9.31it/s]
